In [2]:
from whales_descriptors import do_whales
from rdkit import Chem
import numpy as np
from whales_descriptors import mol_properties
from rdkit.Chem import AllChem
from lcm import domahal, is_rem, docov, lmahal
from do_whales import apply_sign
from do_whales import extract_lcm
import pandas as pd

In [ ]:
suppl = Chem.SDMolSupplier('./Data_and_Output/Task30.sdf')
x, labels = do_whales.main(suppl, charge_threshold=0, do_charge=True, property_name='')

In [ ]:
np.savetxt('./Data_and_Output/Task30_whales_results.txt', x, delimiter=' ', newline='\n') # for descriptors

In [ ]:
#Calcolo whales per recettore nucleare -> covariance matrix su tutti e 
#poi seleziono solo righe e colonne corrispondenti agli atomi della tasca

# Prova con singola molecola da SMILES

In [ ]:
#DA SMILES SERVE OTTIMIZZARE LA STRUTTURA PER AVERE COORDINATE 3D
mol = Chem.MolFromSmiles('C1CCC1OC')
m3 = Chem.AddHs(mol)
#AllChem.EmbedMolecule(m3)
mols = AllChem.EmbedMultipleConfs(m3, 100)
#res = AllChem.MMFFOptimizeMoleculeConfs(m3, maxIters=1000, mmffVariant='MMFF94', )  
res = Chem.rdForceFieldHelpers.MMFFOptimizeMoleculeConfs(m3, maxIters=15000, mmffVariant='MMFF94')  
m3 = Chem.RemoveHs(m3)
print(Chem.MolToMolBlock(m3, confId= res.index(min(res))))  
mol, err = do_whales.import_mol(m3)

In [ ]:
#OPT STABILE
#DA SMILES SERVE OTTIMIZZARE LA STRUTTURA PER AVERE COORDINATE 3D
mol = Chem.MolFromSmiles('C1CCC1OC')
m3 = Chem.AddHs(mol)
#AllChem.EmbedMolecule(m3)
mols = AllChem.EmbedMolecule(m3, randomSeed=26, clearConfs=True, maxAttempts=5)
#mols = AllChem.EmbedMultipleConfs(mols, 100)
#res = AllChem.MMFFOptimizeMoleculeConfs(m3, maxIters=1000, mmffVariant='MMFF94', )  
res = Chem.rdForceFieldHelpers.MMFFOptimizeMolecule(m3, maxIters=15000, mmffVariant='MMFF94')  
m3 = Chem.RemoveHs(m3)
print(Chem.MolToMolBlock(m3))
mol, err = do_whales.import_mol(m3)

In [ ]:
coords, w, err = mol_properties.get_coordinates_and_prop(mol, '', True)
coords

In [ ]:
cov = docov(coords, w)

In [ ]:
n, p = coords.shape

In [ ]:
n,p

In [ ]:
dist = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        d = domahal(i, j, coords, cov)
        dist[i, j] = d / p
dist

In [ ]:
def import_prepare_mol(mol, do_geom):

    # options for sanitization
    san_opt = Chem.SanitizeFlags.SANITIZE_ALL ^ Chem.SanitizeFlags.SANITIZE_KEKULIZE
    # initialization
    err = 0

    # sanitization
    if mol is None:
        err = 1
    else:
        # sanitize
        sanit_fail = Chem.SanitizeMol(mol, catchErrors=True, sanitizeOps=san_opt)
        if sanit_fail:
            raise ValueError(sanit_fail)
            err = 1

        if do_geom is True:
            mol, err = opt_geometry(mol)
    return mol, err


def opt_geometry(mol, max_iter=1000, mmffvariant='MMFF94', seed=26, max_attempts=5): # TODO add in the description - geometry optimization

    # 'mmffVariant : “MMFF94” or “MMFF94s”'
    # seeded coordinate generation, if = -1, no random seed provided
    # removes starting coordinates to ensure reproducibility
    # max attempts, to increase if issues are encountered

    err = 0
    try:
        mol = rdmolops.AddHs(mol)
        AllChem.EmbedMolecule(mol, randomSeed=seed, clearConfs=True, maxAttempts=max_attempts)
        geom = AllChem.MMFFOptimizeMolecule(mol, maxIters=max_iter, mmffVariant=mmffvariant)
        if geom < 0:
               err = 1
    except:
        err = 1

    return mol, err

In [ ]:
mol, err = import_prepare_mol(mol, do_geom=False)

In [ ]:
err

# Prova con un PDB

In [16]:
def extract_idx_bp(path1, path2):
    #import pdb as mol and resave pdb (otherwise indices will not coincide)
    pdb = Chem.rdmolfiles.MolFromPDBFile(path1)
    pdb2 = Chem.rdmolfiles.MolToPDBFile(pdb, "./Data_and_Output/temp_pdb.pdb")    
    pdb_full_df = pd.read_csv("./Data_and_Output/temp_pdb.pdb", delim_whitespace=True, 
                                names=('elix','elix_num', 'res_name', 'res', 'chain',
                                       'res_seq_num','x','y','z','boh','pup','atom'))

    pdb_bp_df = pd.read_csv(path2, delim_whitespace=True, 
                                names=('elix','elix_num', 'res_name', 'res', 'chain',
                                       'res_seq_num','x','y','z','boh','pup','atom'))
    #REMOVE LAST ROW = END
    indexNames = [pdb_full_df[pdb_full_df['elix'] == "TER"].index,
                  pdb_full_df[pdb_full_df['elix'] == "CONECT"].index,
                  pdb_full_df[pdb_full_df['elix'] == "END"].index]
    indexNames = [item for sublist in indexNames for item in sublist]
    pdb_full_df.drop(indexNames , inplace=True)  
    lfull = len(pdb_full_df)
    pdb_bp_df = pdb_bp_df[:-1]
    #select ids of binding pockets (only chain A)
    pdb_full_red = pdb_full_df[pdb_full_df['res_seq_num'].isin(pdb_bp_df['res_seq_num']).values]
    pdb_full_red = pdb_full_red[pdb_full_red['chain']==pdb_bp_df['chain'].values[0]]
    idx_bp=pdb_full_red.index
    return idx_bp, lfull 

In [106]:
def lweucl(x, w):
    # preliminary
    n, p = x.shape  # matrix dimensions

    if len(w) > 0:   # checks whether at least one atom was included
        dist = np.zeros((n, n))  # pre allocation (LCM)

        # do covariance centred on each sample
        #cov = docov(x, w)
        # normalizes partial charges
        den = sum(abs(w))
        if den is 0:
            den = n-1
        w_abs = abs(w)/den
    
        # calculate distance
        for i in range(n):
            for j in range(n):
                d = doweucl(i, j, x, w_abs)
                dist[i, j] = d

        # isolation and remoteness parameters from D
        isol, rem, ir_ratio = is_rem(dist, n)   # calculates atomic parameters from the distance
        res = np.concatenate((rem, isol, ir_ratio), axis=1)   # results concatenation
    else:
        res = np.full((1, 3), -999.0)   # sets missing values

    return res

In [107]:
def doweucl(i, j, x, w_abs):
    d = np.sqrt(np.sum(w_abs[i]*np.power(np.subtract(x[i],x[j]),2)))
    return d

In [113]:
def do_whales_for_pdb(pdb_full, idx_bp, nstep):#CHANGE NSTEP FOR CENTILES (1) INSTEAD OF DECILES (10)
    
    pdbm3 = Chem.RemoveHs(pdb_full)
    mol, err = do_whales.import_mol(pdbm3)
    errors = 0
    
    if nstep == 10:
        lenp = 33
    else:
        lenp = 303

    if err == 1:
        x_all = np.full((lenp,), -999.0)
        errors += err
        print ('Molecule not loaded.')
    else:
        coords, w, err = mol_properties.get_coordinates_and_prop(mol, '_GasteigerCharge', True)
        if err == 0:
            coords_red = coords[idx_bp,:]
            w_red = w[idx_bp,:]
            #####x_all, lab_all = do_lcd(coords_red, w_red)
            #res = lmahal(coords_red, w_red) #with mahalanobis
            res = lweucl(coords_red, w_red)   #with weighted euclidean
            res = apply_sign(w_red, res, 0)
            x_all, lab_all = extract_lcm(res, step = nstep)
            ######
        else:
            x_all = np.full((lenp,), -999.0)  
            errors += 1
            print ('Molecule no computed charges.')    
    descriptors = pd.DataFrame()
    descriptors[str(1)] = x_all
    descriptors = pd.DataFrame.transpose(descriptors)
    descriptors.columns = lab_all
    
    if errors > 0:
        print(str(errors) + 'molecules not loaded/calculated')
    
    return descriptors, lab_all, x_all

In [114]:
idx_bp, lfull = extract_idx_bp("./Data_and_Output/PDBs_FULL_CLEAN/AR_1e3g_A.pdb", "./Data_and_Output/BS_PDBs_PBONLY/AR_1e3g_A.pdb")
idx_bp

Int64Index([  94,   95,   96,   97,   98,   99,  100,  101,  102,  103,
            ...
            1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920],
           dtype='int64', length=823)

In [110]:
pdb = Chem.rdmolfiles.MolFromPDBFile("./Data_and_Output/PDBs_FULL_CLEAN/AR_1e3g_A.pdb")
descriptors, lab_all, x_all = do_whales_for_pdb(pdb, idx_bp, 10)

In [112]:
descriptors

,R_0,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,R_9,...,IR_1,IR_2,IR_3,IR_4,IR_5,IR_6,IR_7,IR_8,IR_9,IR_10
1,-1.424,-0.866,-0.768,-0.679,-0.365,-0.271,-0.157,0.37,0.484,0.659,...,-0.069,-0.046,-0.035,-0.027,-0.021,-0.014,0.025,0.035,0.047,0.473


# BATCH MODE

In [115]:
listPDB = ['AR_1e3g_A.pdb','AR_1z95_A.pdb','AR_2am9_A.pdb','AR_2ama_A.pdb','AR_2ax6_A.pdb','AR_2ax9_A.pdb','AR_2hvc_A.pdb','AR_2ihq_A.pdb','AR_2nw4_A.pdb','AR_2oz7_A.pdb','AR_3b5r_A.pdb','AR_3b65_A.pdb','AR_3b66_A.pdb','AR_3b67_A.pdb','AR_3b68_A.pdb','AR_3g0w_A.pdb','AR_3v49_A.pdb','AR_4ql8_A.pdb','AR_5cj6_A.pdb','AR_5t8e_A.pdb','AR_5t8j_A.pdb','AR_5v8q_A.pdb','ER_1err_A.pdb','ER_1err_B.pdb','ER_1g50_A.pdb','ER_1g50_B.pdb','ER_1g50_C.pdb','ER_1sj0_A.pdb','ER_1uom_A.pdb','ER_1xp1_A.pdb','ER_1xp6_A.pdb','ER_1xp9_A.pdb','ER_1xpc_A.pdb','ER_1xqc_A.pdb','ER_1xqc_B.pdb','ER_1xqc_C.pdb','ER_1xqc_D.pdb','ER_1zky_A.pdb','ER_1zky_B.pdb','ER_2ayr_A.pdb','ER_2b1v_A.pdb','ER_2b1v_B.pdb','ER_2b1z_A.pdb','ER_2b1z_B.pdb','ER_2fai_A.pdb','ER_2fai_B.pdb','ER_2p15_A.pdb','ER_2p15_B.pdb','ER_2pog_A.pdb','ER_2pog_B.pdb','ER_2q70_A.pdb','ER_2q70_B.pdb','ER_2qa8_A.pdb','ER_2qa8_B.pdb','ER_2qe4_A.pdb','ER_2qe4_B.pdb','ER_2qh6_A.pdb','ER_2qh6_B.pdb','ER_2r6w_A.pdb','ER_2r6w_B.pdb','ER_2r6y_A.pdb','ER_2r6y_B.pdb','ER_4mg5_A.pdb','ER_4mg5_B.pdb','ER_4mg6_A.pdb','ER_4mg6_B.pdb','ER_4mg7_A.pdb','ER_4mg7_B.pdb','ER_4mg8_A.pdb','ER_4mg8_B.pdb','ER_4mg9_A.pdb','ER_4mg9_B.pdb','ER_4mga_A.pdb','ER_4mga_B.pdb','ER_4mgb_A.pdb','ER_4mgb_B.pdb','ER_4mgc_A.pdb','ER_4mgc_B.pdb','ER_4mgd_A.pdb','ER_4mgd_B.pdb','ER_5ak2_A.pdb','ER_5ak2_B.pdb','ER_5fqr_A.pdb','ER_5fqs_A.pdb','ER_5t92_A.pdb','ER_5t92_B.pdb','ER_5t97_A.pdb','ER_5t97_B.pdb','ERA_1x7e_A.pdb','ERA_1x7e_B.pdb','ERA_1x7r_A.pdb','ERA_2i0j_A.pdb','ERA_2i0j_B.pdb','ERA_2i0j_C.pdb','ERA_2i0j_D.pdb','ERA_3erd_A.pdb','ERA_3erd_B.pdb','ERA_3ert_A.pdb','ERA_5fqp_A.pdb','ERA_5fqt_A.pdb','ERA_5fqv_A.pdb','ERB_1nde_A.pdb','ERB_1qkn_A.pdb','ERB_1u3q_A.pdb','ERB_1u3q_B.pdb','ERB_1u3q_C.pdb','ERB_1u3q_D.pdb','ERB_1u3r_A.pdb','ERB_1u3r_B.pdb','ERB_1u3s_A.pdb','ERB_1u3s_B.pdb','ERB_1u9e_A.pdb','ERB_1u9e_B.pdb','ERB_1x7b_A.pdb','ERB_1x7b_B.pdb','ERB_1x76_A.pdb','ERB_1x76_B.pdb','ERB_1x78_A.pdb','ERB_1x78_B.pdb','ERB_1yy4_A.pdb','ERB_1yy4_B.pdb','ERB_1yye_A.pdb','ERB_1yye_B.pdb','ERB_1zaf_B.pdb','ERB_2giu_A.pdb','ERB_2i0g_A.pdb','ERB_2i0g_B.pdb','ERB_2j7x_A.pdb','ERB_2jj3_A.pdb','ERB_2jj3_B.pdb','ERB_2nv7_A.pdb','ERB_2nv7_B.pdb','ERB_2qtu_A.pdb','ERB_2qtu_B.pdb','ERB_2z4b_A.pdb','ERB_2z4b_B.pdb','FXR_3dct_A.pdb','FXR_3okh_A.pdb','FXR_3oki_A.pdb','FXR_3oki_C.pdb','FXR_3olf_A.pdb','FXR_3olf_C.pdb','FXR_3omm_A.pdb','FXR_3omm_C.pdb','FXR_3oof_A.pdb','FXR_3oof_C.pdb','FXR_3ook_A.pdb','FXR_3ook_C.pdb','FXR_4oiv_A.pdb','FXR_4oiv_B.pdb','FXR_5q0i_A.pdb','FXR_5q0j_A.pdb','FXR_5q0j_C.pdb','FXR_5q0l_A.pdb','FXR_5q0l_C.pdb','FXR_5q0m_A.pdb','FXR_5q0n_A.pdb','FXR_5q0n_C.pdb','FXR_5q0o_A.pdb','FXR_5q0o_C.pdb','FXR_5q0p_A.pdb','FXR_5q0p_C.pdb','FXR_5q0q_A.pdb','FXR_5q0q_C.pdb','FXR_5q0r_A.pdb','FXR_5q0s_A.pdb','FXR_5q0s_C.pdb','FXR_5q0t_A.pdb','FXR_5q0u_A.pdb','FXR_5q0u_C.pdb','FXR_5q0v_A.pdb','FXR_5q0v_C.pdb','FXR_5q0w_A.pdb','FXR_5q0x_A.pdb','FXR_5q0y_A.pdb','FXR_5q0y_C.pdb','FXR_5q1a_A.pdb','FXR_5q1b_A.pdb','FXR_5q1b_C.pdb','FXR_5q1c_A.pdb','FXR_5q1c_C.pdb','FXR_5q1d_A.pdb','FXR_5q1d_C.pdb','FXR_5q1f_A.pdb','FXR_5q1f_C.pdb','FXR_5q1g_A.pdb','FXR_5q1i_A.pdb','FXR_5q10_A.pdb','FXR_5q11_A.pdb','FXR_5q12_A.pdb','FXR_5q13_A.pdb','FXR_5q13_C.pdb','FXR_5q14_A.pdb','FXR_5q14_C.pdb','FXR_5q15_A.pdb','FXR_5q15_C.pdb','FXR_5q16_A.pdb','FXR_5q16_C.pdb','FXR_5q17_A.pdb','FXR_5q18_A.pdb','FXR_5q18_C.pdb','FXR_5q19_A.pdb','FXR_5q19_C.pdb','GR_1nhz_A.pdb','GR_1p93_A.pdb','GR_1p93_B.pdb','GR_1p93_C.pdb','GR_1p93_D.pdb','GR_3k22_A.pdb','GR_3k22_B.pdb','GR_3k23_A.pdb','GR_3k23_B.pdb','GR_3k23_C.pdb','GR_4csj_A.pdb','GR_4p6w_A.pdb','GR_4p6x_A.pdb','GR_4p6x_C.pdb','GR_4p6x_E.pdb','GR_4p6x_G.pdb','GR_4p6x_I.pdb','GR_4p6x_K.pdb','PPARA_1i7g_A.pdb','PPARA_1kkq_A.pdb','PPARA_1kkq_B.pdb','PPARA_1kkq_C.pdb','PPARA_3fei_A.pdb','PPARA_3g8i_A.pdb','PPARA_3kdt_A.pdb','PPARA_3kdt_B.pdb','PPARA_3kdu_A.pdb','PPARA_3kdu_B.pdb','PPARD_3dy6_A.pdb','PPARD_3dy6_B.pdb','PPARD_3gwx_A.pdb','PPARD_3gwx_B.pdb','PPARD_3gz9_A.pdb','PPARD_3peq_A.pdb','PPARD_3peq_B.pdb','PPARD_3tkm_A.pdb','PPARG_1fm9_D.pdb','PPARG_1i7i_A.pdb','PPARG_1i7i_B.pdb','PPARG_1nyx_A.pdb','PPARG_1zeo_A.pdb','PPARG_2ath_A.pdb','PPARG_2ath_B.pdb','PPARG_2f4b_A.pdb','PPARG_2g0g_A.pdb','PPARG_2g0h_A.pdb','PPARG_2g0h_B.pdb','PPARG_2gtk_A.pdb','PPARG_2hfp_A.pdb','PPARG_2i4j_A.pdb','PPARG_2i4z_A.pdb','PPARG_2p4y_A.pdb','PPARG_2p4y_B.pdb','PPARG_2q8s_A.pdb','PPARG_2yfe_A.pdb','PPARG_2yfe_B.pdb','PPARG_3b1m_A.pdb','PPARG_3fej_A.pdb','PPARG_3fur_A.pdb','PPARG_3g9e_A.pdb','PPARG_3ia6_A.pdb','PPARG_3ia6_B.pdb','PPARG_3lmp_A.pdb','PPARG_3osi_A.pdb','PPARG_3osw_A.pdb','PPARG_3r5n_A.pdb','PPARG_3r8i_A.pdb','PPARG_3sz1_B.pdb','PPARG_3t03_A.pdb','PPARG_3t03_B.pdb','PPARG_3ty0_A.pdb','PPARG_3ty0_B.pdb','PPARG_3u9q_A.pdb','PPARG_4a4v_A.pdb','PPARG_4a4v_B.pdb','PPARG_4a4w_A.pdb','PPARG_4a4w_B.pdb','PPARG_4jaz_A.pdb','PPARG_4prg_A.pdb','PPARG_4prg_B.pdb','PPARG_4prg_C.pdb','PPARG_4prg_D.pdb','PPARG_4r06_A.pdb','PPARG_4xta_A.pdb','PPARG_4xta_B.pdb','PPARG_4xuh_A.pdb','PPARG_4xuh_B.pdb','PPARG_4xum_A.pdb','PPARG_4xum_B.pd','PPARG_4y29_A.pd','PPARG_5f9b_B.pd','PPARG_5lsg_A.pd','PPARG_5two_A.pd','PPARG_5u5l_A.pd','PR_1a28_A.pdb','PR_1a28_B.pdb','PR_1sqn_A.pdb','PR_1sqn_B.pdb','PR_1sr7_A.pdb','PR_1sr7_B.pdb','PR_1zuc_A.pdb','PR_1zuc_B.pdb','PR_2w8y_A.pdb','PR_3g8o_A.pdb','PR_3g8o_B.pdb','PR_3hq5_A.pdb','PR_3hq5_B.pdb','PR_3kba_A.pdb','PR_3kba_B.pdb','PR_4oar_A.pdb','PXR_1m13_A.pdb','PXR_2o9i_A.pdb','PXR_2o9i_B.pdb','RXR_3fal_B.pdb','RXR_3nsq_B.pdb','RXR_3ozj_A.pdb','RXR_3ozj_C.pdb','RXR_3pcu_A.pdb','RXR_3r5m_C.pdb','RXR_4k4j_A.pdb','RXR_4k6i_A.pdb','RXR_4m8e_A.pdb','RXR_4m8h_A.pdb','RXR_4poh_A.pdb','RXR_4poj_A.pdb','RXR_4pp3_A.pdb','RXR_4pp5_A.pdb','RXR_4zsh_A.pdb','RXR_5mkj_A.pdb']
listPDB1 = ['AR_1e3g_A.pdb','AR_1z95_A.pdb','AR_2am9_A.pdb','AR_2ama_A.pdb','AR_2ax6_A.pdb','AR_2ax9_A.pdb','AR_2hvc_A.pdb','AR_2ihq_A.pdb','AR_2nw4_A.pdb','AR_2oz7_A.pdb','AR_3b5r_A.pdb','AR_3b65_A.pdb','AR_3b66_A.pdb','AR_3b67_A.pdb','AR_3b68_A.pdb','AR_3g0w_A.pdb','AR_3v49_A.pdb','AR_4ql8_A.pdb','AR_5cj6_A.pdb','AR_5t8e_A.pdb','AR_5t8j_A.pdb','AR_5v8q_A.pdb','ER_1err_A.pdb','ER_1err_B.pdb','ER_1g50_A.pdb','ER_1g50_B.pdb','ER_1g50_C.pdb','ER_1sj0_A.pdb','ER_1uom_A.pdb','ER_1xp1_A.pdb','ER_1xp6_A.pdb','ER_1xp9_A.pdb','ER_1xpc_A.pdb','ER_1xqc_A.pdb','ER_1xqc_B.pdb','ER_1xqc_C.pdb','ER_1xqc_D.pdb','ER_1zky_A.pdb','ER_1zky_B.pdb','ER_2ayr_A.pdb','ER_2b1v_A.pdb','ER_2b1v_B.pdb','ER_2b1z_A.pdb','ER_2b1z_B.pdb','ER_2fai_A.pdb','ER_2fai_B.pdb','ER_2p15_A.pdb','ER_2p15_B.pdb','ER_2pog_A.pdb','ER_2pog_B.pdb']
listPDB2 = ['ER_2q70_A.pdb','ER_2q70_B.pdb','ER_2qa8_A.pdb','ER_2qa8_B.pdb','ER_2qe4_A.pdb','ER_2qe4_B.pdb','ER_2qh6_A.pdb','ER_2qh6_B.pdb','ER_2r6w_A.pdb','ER_2r6w_B.pdb','ER_2r6y_A.pdb','ER_2r6y_B.pdb','ER_4mg5_A.pdb','ER_4mg5_B.pdb','ER_4mg6_A.pdb','ER_4mg6_B.pdb','ER_4mg7_A.pdb','ER_4mg7_B.pdb','ER_4mg8_A.pdb','ER_4mg8_B.pdb','ER_4mg9_A.pdb','ER_4mg9_B.pdb','ER_4mga_A.pdb','ER_4mga_B.pdb','ER_4mgb_A.pdb','ER_4mgb_B.pdb','ER_4mgc_A.pdb','ER_4mgc_B.pdb','ER_4mgd_A.pdb','ER_4mgd_B.pdb','ER_5ak2_A.pdb','ER_5ak2_B.pdb','ER_5fqr_A.pdb','ER_5fqs_A.pdb','ER_5t92_A.pdb','ER_5t92_B.pdb','ER_5t97_A.pdb','ER_5t97_B.pdb','ERA_1x7e_A.pdb','ERA_1x7e_B.pdb','ERA_1x7r_A.pdb','ERA_2i0j_A.pdb','ERA_2i0j_B.pdb','ERA_2i0j_C.pdb','ERA_2i0j_D.pdb','ERA_3erd_A.pdb','ERA_3erd_B.pdb','ERA_3ert_A.pdb','ERA_5fqp_A.pdb','ERA_5fqt_A.pdb','ERA_5fqv_A.pdb','ERB_1nde_A.pdb','ERB_1qkn_A.pdb','ERB_1u3q_A.pdb','ERB_1u3q_B.pdb','ERB_1u3q_C.pdb','ERB_1u3q_D.pdb','ERB_1u3r_A.pdb','ERB_1u3r_B.pdb','ERB_1u3s_A.pdb','ERB_1u3s_B.pdb','ERB_1u9e_A.pdb','ERB_1u9e_B.pdb','ERB_1x7b_A.pdb','ERB_1x7b_B.pdb','ERB_1x76_A.pdb','ERB_1x76_B.pdb','ERB_1x78_A.pdb','ERB_1x78_B.pdb','ERB_1yy4_A.pdb','ERB_1yy4_B.pdb','ERB_1yye_A.pdb','ERB_1yye_B.pdb']
listPDB3 = ['ERB_1zaf_B.pdb','ERB_2giu_A.pdb','ERB_2i0g_A.pdb','ERB_2i0g_B.pdb','ERB_2j7x_A.pdb','ERB_2jj3_A.pdb','ERB_2jj3_B.pdb','ERB_2nv7_A.pdb','ERB_2nv7_B.pdb','ERB_2qtu_A.pdb','ERB_2qtu_B.pdb','ERB_2z4b_A.pdb','ERB_2z4b_B.pdb','FXR_3dct_A.pdb','FXR_3okh_A.pdb','FXR_3oki_A.pdb','FXR_3oki_C.pdb','FXR_3olf_A.pdb','FXR_3olf_C.pdb','FXR_3omm_A.pdb','FXR_3omm_C.pdb','FXR_3oof_A.pdb','FXR_3oof_C.pdb','FXR_3ook_A.pdb','FXR_3ook_C.pdb','FXR_4oiv_A.pdb','FXR_4oiv_B.pdb','FXR_5q0i_A.pdb','FXR_5q0j_A.pdb','FXR_5q0j_C.pdb','FXR_5q0l_A.pdb','FXR_5q0l_C.pdb','FXR_5q0m_A.pdb','FXR_5q0n_A.pdb','FXR_5q0n_C.pdb','FXR_5q0o_A.pdb','FXR_5q0o_C.pdb','FXR_5q0p_A.pdb','FXR_5q0p_C.pdb','FXR_5q0q_A.pdb','FXR_5q0q_C.pdb','FXR_5q0r_A.pdb','FXR_5q0s_A.pdb','FXR_5q0s_C.pdb','FXR_5q0t_A.pdb','FXR_5q0u_A.pdb','FXR_5q0u_C.pdb','FXR_5q0v_A.pdb','FXR_5q0v_C.pdb','FXR_5q0w_A.pdb','FXR_5q0x_A.pdb','FXR_5q0y_A.pdb','FXR_5q0y_C.pdb','FXR_5q1a_A.pdb']
listPDB4 = ['FXR_5q1b_A.pdb','FXR_5q1b_C.pdb','FXR_5q1c_A.pdb','FXR_5q1c_C.pdb','FXR_5q1d_A.pdb','FXR_5q1d_C.pdb','FXR_5q1f_A.pdb','FXR_5q1f_C.pdb','FXR_5q1g_A.pdb','FXR_5q1i_A.pdb','FXR_5q10_A.pdb','FXR_5q11_A.pdb','FXR_5q12_A.pdb','FXR_5q13_A.pdb','FXR_5q13_C.pdb','FXR_5q14_A.pdb','FXR_5q14_C.pdb','FXR_5q15_A.pdb','FXR_5q15_C.pdb','FXR_5q16_A.pdb','FXR_5q16_C.pdb','FXR_5q17_A.pdb','FXR_5q18_A.pdb','FXR_5q18_C.pdb','FXR_5q19_A.pdb','FXR_5q19_C.pdb','GR_1nhz_A.pdb','GR_1p93_A.pdb','GR_1p93_B.pdb','GR_1p93_C.pdb','GR_1p93_D.pdb','GR_3k22_A.pdb','GR_3k22_B.pdb','GR_3k23_A.pdb','GR_3k23_B.pdb','GR_3k23_C.pdb','GR_4csj_A.pdb','GR_4p6w_A.pdb','GR_4p6x_A.pdb','GR_4p6x_C.pdb','GR_4p6x_E.pdb','GR_4p6x_G.pdb','GR_4p6x_I.pdb','GR_4p6x_K.pdb','PPARA_1i7g_A.pdb','PPARA_1kkq_A.pdb','PPARA_1kkq_B.pdb','PPARA_1kkq_C.pdb']
listPDB5 = ['PPARA_3fei_A.pdb','PPARA_3g8i_A.pdb','PPARA_3kdt_A.pdb','PPARA_3kdt_B.pdb','PPARA_3kdu_A.pdb','PPARA_3kdu_B.pdb','PPARD_3dy6_A.pdb','PPARD_3dy6_B.pdb','PPARD_3gwx_A.pdb','PPARD_3gwx_B.pdb','PPARD_3gz9_A.pdb','PPARD_3peq_A.pdb','PPARD_3peq_B.pdb','PPARD_3tkm_A.pdb','PPARG_1fm9_D.pdb','PPARG_1i7i_A.pdb','PPARG_1i7i_B.pdb','PPARG_1nyx_A.pdb','PPARG_1zeo_A.pdb','PPARG_2ath_A.pdb','PPARG_2ath_B.pdb','PPARG_2f4b_A.pdb','PPARG_2g0g_A.pdb','PPARG_2g0h_A.pdb','PPARG_2g0h_B.pdb','PPARG_2gtk_A.pdb','PPARG_2hfp_A.pdb','PPARG_2i4j_A.pdb','PPARG_2i4z_A.pdb','PPARG_2p4y_A.pdb','PPARG_2p4y_B.pdb','PPARG_2q8s_A.pdb','PPARG_2yfe_A.pdb','PPARG_2yfe_B.pdb','PPARG_3b1m_A.pdb','PPARG_3fej_A.pdb','PPARG_3fur_A.pdb','PPARG_3g9e_A.pdb','PPARG_3ia6_A.pdb','PPARG_3ia6_B.pdb','PPARG_3lmp_A.pdb','PPARG_3osi_A.pdb','PPARG_3osw_A.pdb','PPARG_3r5n_A.pdb','PPARG_3r8i_A.pdb','PPARG_3sz1_B.pdb','PPARG_3t03_A.pdb','PPARG_3t03_B.pdb','PPARG_3ty0_A.pdb','PPARG_3ty0_B.pdb','PPARG_3u9q_A.pdb','PPARG_4a4v_A.pdb','PPARG_4a4v_B.pdb','PPARG_4a4w_A.pdb','PPARG_4a4w_B.pdb','PPARG_4jaz_A.pdb','PPARG_4prg_A.pdb','PPARG_4prg_B.pdb','PPARG_4prg_C.pdb','PPARG_4prg_D.pdb','PPARG_4r06_A.pdb','PPARG_4xta_A.pdb','PPARG_4xta_B.pdb','PPARG_4xuh_A.pdb','PPARG_4xuh_B.pdb','PPARG_4xum_A.pdb']
listPDB6 = ['PPARG_4xum_B.pdb','PPARG_4y29_A.pdb','PPARG_5f9b_B.pdb','PPARG_5lsg_A.pdb','PPARG_5two_A.pdb','PPARG_5u5l_A.pdb','PR_1a28_A.pdb','PR_1a28_B.pdb','PR_1sqn_A.pdb','PR_1sqn_B.pdb','PR_1sr7_A.pdb','PR_1sr7_B.pdb','PR_1zuc_A.pdb','PR_1zuc_B.pdb','PR_2w8y_A.pdb','PR_3g8o_A.pdb','PR_3g8o_B.pdb','PR_3hq5_A.pdb','PR_3hq5_B.pdb','PR_3kba_A.pdb','PR_3kba_B.pdb','PR_4oar_A.pdb']
listPDB7 = ['PXR_1m13_A.pdb','PXR_2o9i_A.pdb','PXR_2o9i_B.pdb','RXR_3fal_B.pdb','RXR_3nsq_B.pdb','RXR_3ozj_A.pdb','RXR_3ozj_C.pdb','RXR_3pcu_A.pdb']
listPDB8 = ['RXR_3r5m_C.pdb','RXR_4k4j_A.pdb','RXR_4k6i_A.pdb','RXR_4m8e_A.pdb','RXR_4m8h_A.pdb','RXR_4poh_A.pdb','RXR_4poj_A.pdb','RXR_4pp3_A.pdb','RXR_4pp5_A.pdb','RXR_4zsh_A.pdb','RXR_5mkj_A.pdb']

listPDB_set = [['AR_1e3g_A.pdb','AR_1z95_A.pdb','AR_2am9_A.pdb','AR_2ama_A.pdb','AR_2ax6_A.pdb','AR_2ax9_A.pdb','AR_2hvc_A.pdb','AR_2ihq_A.pdb','AR_2nw4_A.pdb','AR_2oz7_A.pdb','AR_3b5r_A.pdb','AR_3b65_A.pdb','AR_3b66_A.pdb','AR_3b67_A.pdb','AR_3b68_A.pdb','AR_3g0w_A.pdb','AR_3v49_A.pdb','AR_4ql8_A.pdb','AR_5cj6_A.pdb','AR_5t8e_A.pdb','AR_5t8j_A.pdb','AR_5v8q_A.pdb'],['ER_1err_A.pdb','ER_1err_B.pdb','ER_1g50_A.pdb','ER_1g50_B.pdb','ER_1g50_C.pdb','ER_1sj0_A.pdb','ER_1uom_A.pdb','ER_1xp1_A.pdb','ER_1xp6_A.pdb','ER_1xp9_A.pdb','ER_1xpc_A.pdb','ER_1xqc_A.pdb','ER_1xqc_B.pdb','ER_1xqc_C.pdb','ER_1xqc_D.pdb','ER_1zky_A.pdb','ER_1zky_B.pdb','ER_2ayr_A.pdb','ER_2b1v_A.pdb','ER_2b1v_B.pdb','ER_2b1z_A.pdb','ER_2b1z_B.pdb','ER_2fai_A.pdb','ER_2fai_B.pdb','ER_2p15_A.pdb','ER_2p15_B.pdb','ER_2pog_A.pdb','ER_2pog_B.pdb','ER_2q70_A.pdb','ER_2q70_B.pdb','ER_2qa8_A.pdb','ER_2qa8_B.pdb','ER_2qe4_A.pdb','ER_2qe4_B.pdb','ER_2qh6_A.pdb','ER_2qh6_B.pdb','ER_2r6w_A.pdb','ER_2r6w_B.pdb','ER_2r6y_A.pdb','ER_2r6y_B.pdb','ER_4mg5_A.pdb','ER_4mg5_B.pdb','ER_4mg6_A.pdb','ER_4mg6_B.pdb','ER_4mg7_A.pdb','ER_4mg7_B.pdb','ER_4mg8_A.pdb','ER_4mg8_B.pdb','ER_4mg9_A.pdb','ER_4mg9_B.pdb','ER_4mga_A.pdb','ER_4mga_B.pdb','ER_4mgb_A.pdb','ER_4mgb_B.pdb','ER_4mgc_A.pdb','ER_4mgc_B.pdb','ER_4mgd_A.pdb','ER_4mgd_B.pdb','ER_5ak2_A.pdb','ER_5ak2_B.pdb','ER_5fqr_A.pdb','ER_5fqs_A.pdb','ER_5t92_A.pdb','ER_5t92_B.pdb','ER_5t97_A.pdb','ER_5t97_B.pdb'],['ERA_1x7e_A.pdb','ERA_1x7e_B.pdb','ERA_1x7r_A.pdb','ERA_2i0j_A.pdb','ERA_2i0j_B.pdb','ERA_2i0j_C.pdb','ERA_2i0j_D.pdb','ERA_3erd_A.pdb','ERA_3erd_B.pdb','ERA_3ert_A.pdb','ERA_5fqp_A.pdb','ERA_5fqt_A.pdb','ERA_5fqv_A.pdb'],['ERB_1nde_A.pdb','ERB_1qkn_A.pdb','ERB_1u3q_A.pdb','ERB_1u3q_B.pdb','ERB_1u3q_C.pdb','ERB_1u3q_D.pdb','ERB_1u3r_A.pdb','ERB_1u3r_B.pdb','ERB_1u3s_A.pdb','ERB_1u3s_B.pdb','ERB_1u9e_A.pdb','ERB_1u9e_B.pdb','ERB_1x7b_A.pdb','ERB_1x7b_B.pdb','ERB_1x76_A.pdb','ERB_1x76_B.pdb','ERB_1x78_A.pdb','ERB_1x78_B.pdb','ERB_1yy4_A.pdb','ERB_1yy4_B.pdb','ERB_1yye_A.pdb','ERB_1yye_B.pdb','ERB_1zaf_B.pdb','ERB_2giu_A.pdb','ERB_2i0g_A.pdb','ERB_2i0g_B.pdb','ERB_2j7x_A.pdb','ERB_2jj3_A.pdb','ERB_2jj3_B.pdb','ERB_2nv7_A.pdb','ERB_2nv7_B.pdb','ERB_2qtu_A.pdb','ERB_2qtu_B.pdb','ERB_2z4b_A.pdb','ERB_2z4b_B.pdb'],['FXR_3dct_A.pdb','FXR_3okh_A.pdb','FXR_3oki_A.pdb','FXR_3oki_C.pdb','FXR_3olf_A.pdb','FXR_3olf_C.pdb','FXR_3omm_A.pdb','FXR_3omm_C.pdb','FXR_3oof_A.pdb','FXR_3oof_C.pdb','FXR_3ook_A.pdb','FXR_3ook_C.pdb','FXR_4oiv_A.pdb','FXR_4oiv_B.pdb','FXR_5q0i_A.pdb','FXR_5q0j_A.pdb','FXR_5q0j_C.pdb','FXR_5q0l_A.pdb','FXR_5q0l_C.pdb','FXR_5q0m_A.pdb','FXR_5q0n_A.pdb','FXR_5q0n_C.pdb','FXR_5q0o_A.pdb','FXR_5q0o_C.pdb','FXR_5q0p_A.pdb','FXR_5q0p_C.pdb','FXR_5q0q_A.pdb','FXR_5q0q_C.pdb','FXR_5q0r_A.pdb','FXR_5q0s_A.pdb','FXR_5q0s_C.pdb','FXR_5q0t_A.pdb','FXR_5q0u_A.pdb','FXR_5q0u_C.pdb','FXR_5q0v_A.pdb','FXR_5q0v_C.pdb','FXR_5q0w_A.pdb','FXR_5q0x_A.pdb','FXR_5q0y_A.pdb','FXR_5q0y_C.pdb','FXR_5q1a_A.pdb','FXR_5q1b_A.pdb','FXR_5q1b_C.pdb','FXR_5q1c_A.pdb','FXR_5q1c_C.pdb','FXR_5q1d_A.pdb','FXR_5q1d_C.pdb','FXR_5q1f_A.pdb','FXR_5q1f_C.pdb','FXR_5q1g_A.pdb','FXR_5q1i_A.pdb','FXR_5q10_A.pdb','FXR_5q11_A.pdb','FXR_5q12_A.pdb','FXR_5q13_A.pdb','FXR_5q13_C.pdb','FXR_5q14_A.pdb','FXR_5q14_C.pdb','FXR_5q15_A.pdb','FXR_5q15_C.pdb','FXR_5q16_A.pdb','FXR_5q16_C.pdb','FXR_5q17_A.pdb','FXR_5q18_A.pdb','FXR_5q18_C.pdb','FXR_5q19_A.pdb','FXR_5q19_C.pdb'],['GR_1nhz_A.pdb','GR_1p93_A.pdb','GR_1p93_B.pdb','GR_1p93_C.pdb','GR_1p93_D.pdb','GR_3k22_A.pdb','GR_3k22_B.pdb','GR_3k23_A.pdb','GR_3k23_B.pdb','GR_3k23_C.pdb','GR_4csj_A.pdb','GR_4p6w_A.pdb','GR_4p6x_A.pdb','GR_4p6x_C.pdb','GR_4p6x_E.pdb','GR_4p6x_G.pdb','GR_4p6x_I.pdb','GR_4p6x_K.pdb'],['PPARA_1i7g_A.pdb','PPARA_1kkq_A.pdb','PPARA_1kkq_B.pdb','PPARA_1kkq_C.pdb','PPARA_3fei_A.pdb','PPARA_3g8i_A.pdb','PPARA_3kdt_A.pdb','PPARA_3kdt_B.pdb','PPARA_3kdu_A.pdb','PPARA_3kdu_B.pdb'],['PPARD_3dy6_A.pdb','PPARD_3dy6_B.pdb','PPARD_3gwx_A.pdb','PPARD_3gwx_B.pdb','PPARD_3gz9_A.pdb','PPARD_3peq_A.pdb','PPARD_3peq_B.pdb','PPARD_3tkm_A.pdb'],['PPARG_1fm9_D.pdb','PPARG_1i7i_A.pdb','PPARG_1i7i_B.pdb','PPARG_1nyx_A.pdb','PPARG_1zeo_A.pdb','PPARG_2ath_A.pdb','PPARG_2ath_B.pdb','PPARG_2f4b_A.pdb','PPARG_2g0g_A.pdb','PPARG_2g0h_A.pdb','PPARG_2g0h_B.pdb','PPARG_2gtk_A.pdb','PPARG_2hfp_A.pdb','PPARG_2i4j_A.pdb','PPARG_2i4z_A.pdb','PPARG_2p4y_A.pdb','PPARG_2p4y_B.pdb','PPARG_2q8s_A.pdb','PPARG_2yfe_A.pdb','PPARG_2yfe_B.pdb','PPARG_3b1m_A.pdb','PPARG_3fej_A.pdb','PPARG_3fur_A.pdb','PPARG_3g9e_A.pdb','PPARG_3ia6_A.pdb','PPARG_3ia6_B.pdb','PPARG_3lmp_A.pdb','PPARG_3osi_A.pdb','PPARG_3osw_A.pdb','PPARG_3r5n_A.pdb','PPARG_3r8i_A.pdb','PPARG_3sz1_B.pdb','PPARG_3t03_A.pdb','PPARG_3t03_B.pdb','PPARG_3ty0_A.pdb','PPARG_3ty0_B.pdb','PPARG_3u9q_A.pdb','PPARG_4a4v_A.pdb','PPARG_4a4v_B.pdb','PPARG_4a4w_A.pdb','PPARG_4a4w_B.pdb','PPARG_4jaz_A.pdb','PPARG_4prg_A.pdb','PPARG_4prg_B.pdb','PPARG_4prg_C.pdb','PPARG_4prg_D.pdb','PPARG_4r06_A.pdb','PPARG_4xta_A.pdb','PPARG_4xta_B.pdb','PPARG_4xuh_A.pdb','PPARG_4xuh_B.pdb','PPARG_4xum_A.pdb','PPARG_4xum_B.pd','PPARG_4y29_A.pd','PPARG_5f9b_B.pd','PPARG_5lsg_A.pd','PPARG_5two_A.pd','PPARG_5u5l_A.pd'],['PR_1a28_A.pdb','PR_1a28_B.pdb','PR_1sqn_A.pdb','PR_1sqn_B.pdb','PR_1sr7_A.pdb','PR_1sr7_B.pdb','PR_1zuc_A.pdb','PR_1zuc_B.pdb','PR_2w8y_A.pdb','PR_3g8o_A.pdb','PR_3g8o_B.pdb','PR_3hq5_A.pdb','PR_3hq5_B.pdb','PR_3kba_A.pdb','PR_3kba_B.pdb','PR_4oar_A.pdb'],['PXR_1m13_A.pdb','PXR_2o9i_A.pdb','PXR_2o9i_B.pdb'],['RXR_3fal_B.pdb','RXR_3nsq_B.pdb','RXR_3ozj_A.pdb','RXR_3ozj_C.pdb','RXR_3pcu_A.pdb','RXR_3r5m_C.pdb','RXR_4k4j_A.pdb','RXR_4k6i_A.pdb','RXR_4m8e_A.pdb','RXR_4m8h_A.pdb','RXR_4poh_A.pdb','RXR_4poj_A.pdb','RXR_4pp3_A.pdb','RXR_4pp5_A.pdb','RXR_4zsh_A.pdb','RXR_5mkj_A.pdb']]

path_full = "./Data_and_Output/PDBs_FULL_CLEAN/"
path_bp = "./Data_and_Output/BS_PDBs_PBONLY/"
len(listPDB1),len(listPDB2),len(listPDB3),len(listPDB4),len(listPDB5),len(listPDB6),len(listPDB7),len(listPDB8), len(listPDB)

(50, 73, 54, 48, 66, 22, 8, 11, 332)

In [116]:
50+73+54+48+66+22+8+11

332

In [ ]:
import time
start_time = time.time()

all_whales = []

for x in listPDB:  #should be listPDB but split for avoid errors...
    print(x)
    path_pdb_full = path_full + x
    path_pdb_bp = path_bp + x
    pdb = Chem.rdmolfiles.MolFromPDBFile(path_pdb_full)
    idx, lfull = extract_idx_bp(path_pdb_full, path_pdb_bp)
    
    if pdb.GetNumAtoms()==lfull:
        descriptors, lab_all, x_all = do_whales_for_pdb(pdb, idx, 10)
        all_whales.append(x_all)
        np.savetxt(r'./Data_and_Output/ALLWHALEStemp_EUCLID1.txt', all_whales)
    else:
        print("lenghts don't match: len rdkitpdb: " + str(pdb.GetNumAtoms()) + " len df: " + str(lfull))
    
print("--- %s seconds ---" % (time.time() - start_time))

AR_1e3g_A.pdb
AR_1z95_A.pdb
AR_2am9_A.pdb
